In [9]:
from util.utils import *
import numpy as np

In [10]:
def orderByIDandTime(df):
    # Ordena o DataFrame com base nas colunas 'flightid' e 'dt_radar'
    df = df.sort_values(by=['flightid', 'dt_radar'])
    return df

def convertToDegrees(coordinateColumn):
    # Converte valores de radianos para graus
    coordinateColumn = coordinateColumn * 180 / np.pi
    return coordinateColumn

def convertToRadians(coordinateColumn):
    # Converte valores de graus para radianos
    coordinateColumn = coordinateColumn * np.pi / 180
    return coordinateColumn

def calculateDistance(lat1_deg, lon1_deg, lat2_deg, lon2_deg):
    # Calcula a distância entre dois pontos geográficos usando a fórmula de Haversine
    R = 6371e3  # Raio da Terra em metros
    lat1_rad = convertToRadians(lat1_deg)
    lon1_rad = convertToRadians(lon1_deg)
    lat2_rad = convertToRadians(lat2_deg)
    lon2_rad = convertToRadians(lon2_deg)

    delta_lat_rad = lat2_rad - lat1_rad
    delta_lon_rad = lon2_rad - lon1_rad

    a = np.sin(delta_lat_rad/2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(delta_lon_rad/2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    distance_meters = R * c
    return distance_meters

def calculateBearing(lat1_deg, lon1_deg, lat2_deg, lon2_deg):
    # Calcula o rumo (bearing) entre dois pontos geográficos
    lat1_rad = convertToRadians(lat1_deg)
    lon1_rad = convertToRadians(lon1_deg)
    lat2_rad = convertToRadians(lat2_deg)
    lon2_rad = convertToRadians(lon2_deg)

    delta_lon_rad = lon2_rad - lon1_rad

    y = np.sin(delta_lon_rad) * np.cos(lat2_rad)
    x = np.cos(lat1_rad) * np.sin(lat2_rad) - np.sin(lat1_rad) * np.cos(lat2_rad) * np.cos(delta_lon_rad)

    theta_rad = np.arctan2(y, x)
    bearing_deg = (theta_rad * 180 / np.pi + 360) % 360
    return bearing_deg

def calculateSpeed(distance_meters, time_seconds):
    # Calcula a velocidade em metros por segundo com base na distância e no tempo
    speed_mps = distance_meters / time_seconds
    return speed_mps

def calculateTime(time1, time2):
    # Calcula o tempo em segundos entre duas instâncias de tempo
    time_seconds = (time2 - time1) / np.timedelta64(1, 's')
    return time_seconds

## Calcula a distância entre as linhas consecutivas do DataFrame.
def calculateDistanceBetweenRows(df):
    ## Calcula a distância em metros entre a latitude e longitude da linha atual e a latitude e longitude da linha seguinte (usando .shift(-1) para pegar a linha seguinte).
    df['distance_m'] = calculateDistance(df['lat_deg'], df['lon_deg'], df['lat_deg'].shift(-1), df['lon_deg'].shift(-1))
    return df

def calculateBearingBetweenRows(df):
    ## Calcula o rumo em graus entre a latitude e longitude da linha atual e a latitude e longitude da linha seguinte (usando .shift(-1) para pegar a linha seguinte).
    df['bearing_deg'] = calculateBearing(df['lat_deg'], df['lon_deg'], df['lat_deg'].shift(-1), df['lon_deg'].shift(-1))
    return df

def calculateSpeedBetweenRows(df):
    #df['time_hours'] = df['time_seconds'] * 3600
    ## Calcula a velocidade média em metros por segundo entre a linha atual e a linha seguinte.
    df['speed_mps'] = calculateSpeed(df['distance_m'], df['time_seconds'])
    return df

def calculateTimeBetweenRows(df):
    ## Converte a coluna "dt_radar" em objetos de data/hora, garantindo que ela seja do tipo datetime.
    df['dt_radar'] = pd.to_datetime(df['dt_radar'])
    ## Calcula a diferença em segundos entre a linha atual e a linha seguinte na coluna "dt_radar" (usando .shift(-1) para pegar a linha seguinte).
    df['time_seconds'] = (df['dt_radar'].shift(-1) - df['dt_radar']).dt.total_seconds()
    return df


def convertCoordinatesToDegrees(df):
    ## convert lat_deg and lon_deg from radians to degrees
    df['lat'] = convertToDegrees(df['lat'])
    df['lon'] = convertToDegrees(df['lon'])

    return df

def convertKnotsToMetersPerSecond(df):
    ## converte speed_knots para speed_mps
    df['speed'] = df['speed'] * 0.514444444
    return df

def convertFeetToMeters(df):
    ## converte flightlevel_ft para flightlevel_m
    df['flightlevel'] = df['flightlevel'] * 0.3048
    return df

def renameColumns(df):
    ## rename lat, lon, speed and flightlevel columns to lat_deg, lon_deg, speed_ and flightlevel_ft
    df = df.rename(columns={'lat': 'lat_deg', 
                            'lon': 'lon_deg', 
                            'speed': 'speed_mps', 
                            'flightlevel': 'flightlevel_m'})
    return df


In [11]:
def cat62_dataPipeline():
    df = openDataframe("cat62_silver.csv")
    df = removeDuplicatesRows(df)
    df = orderByIDandTime(df)
    df = convertCoordinatesToDegrees(df)
    df = convertKnotsToMetersPerSecond(df)
    df = convertFeetToMeters(df)
    df = renameColumns(df)
    df = calculateDistanceBetweenRows(df)
    df = calculateBearingBetweenRows(df)
    df = calculateTimeBetweenRows(df)
    #df = calculateSpeedBetweenRows(df)

    #saveDataframe(df, "cat62_silver.csv")   
    return df

df = cat62_dataPipeline()
df.head()

,flightid,lat_deg,lon_deg,flightlevel_m,speed_mps,dt_radar,dt_radar_formated,distance_m,bearing_deg,time_seconds
427,097eebc964ab2e3833e2d839359e9592,-23.414552,-46.434982,0.0000,85.912222,1970-01-01 00:27:52.534200,"Saturday, December 31, 2022 09:50:00",343.523232,44.750734,0.000004
420,097eebc964ab2e3833e2d839359e9592,-23.412358,-46.432612,0.0000,87.455555,1970-01-01 00:27:52.534204,"Saturday, December 31, 2022 09:50:04",4528.173785,37.105017,0.000056
422,097eebc964ab2e3833e2d839359e9592,-23.379878,-46.405847,15.8496,106.490000,1970-01-01 00:27:52.534260,"Saturday, December 31, 2022 09:51:00",1942.201111,38.457495,0.000004
418,097eebc964ab2e3833e2d839359e9592,-23.366200,-46.394013,16.4592,112.663333,1970-01-01 00:27:52.534264,"Saturday, December 31, 2022 09:51:04",6299.437355,16.152442,0.000056
415,097eebc964ab2e3833e2d839359e9592,-23.311783,-46.376852,23.7744,136.327778,1970-01-01 00:27:52.534320,"Saturday, December 31, 2022 09:52:00",585.016123,319.697236,0.000004


* `distance_m (Distância em metros)`:

A função `calculateDistance(lat1_deg, lon1_deg, lat2_deg, lon2_deg)` calcula a distância em metros entre dois pontos geográficos (latitude e longitude) usando a fórmula da haversine.
Os valores de latitude e longitude (lat e lon) são passados como argumentos para essa função.
A fórmula da haversine considera a curvatura da Terra para calcular a distância real entre dois pontos.

* `bearing_deg (Direção em graus)`:

A função `calculateBearing(lat1_deg, lon1_deg, lat2_deg, lon2_deg)` calcula a direção (rumo) em graus de um ponto 1 para um ponto 2.
Os valores de latitude e longitude (lat e lon) são passados como argumentos para essa função.
A função utiliza fórmulas trigonométricas para calcular o ângulo entre os dois pontos.

* `time_seconds (Tempo em segundos)`:

A `função calculateTime(time1, time2)` calcula a diferença de tempo em segundos entre dois valores de data/hora (time1 e time2).
A coluna dt_radar é convertida em objetos de data/hora usando pd.to_datetime e, em seguida, a diferença de tempo é calculada em segundos usando dt.total_seconds().

* speed_mps (Velocidade em metros por segundo):

A função `calculateSpeed(distance_meters, time_seconds)` calcula a velocidade média em metros por segundo.
A distância em metros (distance_meters) e o tempo em segundos (time_seconds) são passados como argumentos para essa função.
A velocidade é calculada dividindo a distância pelo tempo
